In [1]:
# ATENÇÃO À UTILIZAÇÃO DESTA FUNÇÃO
# DEMASIADAS QUERIES SEGUIDAS PODEM LEVAR A BLOQUEIO POR PARTE DO SERVIDOR
# EM PRINCÍPIO O MÓDULO ENTREZ ESTÁ PREPARADO PARA LIDAR COM ISSO MAS CONVÉM TER SEMPRE CUIDADO

def pesquisa_ncbi(email, term, db = 'pubmed', retmax = 10, rettype = 'abstract', retmode = 'text', display = 'Y' , save = 'N', ext='txt'):
    '''
    NOTA - PARA USAR RETMODE XML TÊM DE SER DECLARADA UMA NOVA VARIÁVEL POIS A FUNÇÃO AINDA NÃO CRIA FICHEIROS XML
    
    Função para a pesquisa de bibliografia na base de dados NCBI utilizando BioPython
    Esta função contempla um conjunto de parâmetros para seleção de base de dados e outputs desejado adaptando-se
    ao objetivo da pesquisa. 

    Por defeito a função pesquisa a base de dados pubmed e capta os 10 primeiros resultados para uma pesquisa
    generalizada. 

    Tem a opção de gerar um ficheiro de texto com os resultados da pesquisa de artigos e respetivos abstracts.

    No entanto a função pode ser especializada para uma pesquisa em base de dados de genes para captar ficheiros
    GenBank e FASTA para posterior análise com as funções específicas que estamos a desenvolver.

    Parâmetros
    ----------

    email : str
        email a ser utilizado pelo módulo Entrez para  pesquisa (e.g. exempl@mail.com)

    term : str
        termo a ser utilizado na pesquisa (e.g. 'gene-123 functions')

    db  : str
        base de dados onde irá ser realizada a pesquisa. Por defeito é utilizada a 'pubmed'
        Para efeitos deste trabalho iremos também utilizar: 'gene', 'nuccore' e 'protein'
        Outras opções disponíveis podem ser encontradas aqui: https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    retmax : int
        define o número de resultados que queremos retirar para o output final (10 por defeito)

    rettype : str
        define o tipo de resultado que pretendemos retirar da base de dados ('abstract' por defeito)
        pode ser mudado para 'gb' para base de dados 'gene' ou 'fasta' para base de dados de nucleotidos ou proteina (sequencias)
        
    retmode : str
        define o retmode para a captura de resultados da pesquisa na base de dados. Por defeito 
        o retmode é text mas pode ser adaptado para 'xml' quando é pretendido obter um ficheiro xml (e.g base de dados nucleotide, ou genbank)

    display : str
        define se é pretendido o output da função legível (útil para quando se quer refinar a pesquisa antes/sem gravar o ficheiro)
        por defeito apresenta o resultado em formato legível
    
    save : str
        define se é pretendido gravar ou não o ficheiro de resultados na pasta 'lit-search-output'
        por defeito este parâmetro está inactivo

            
    
    '''
    # TODO criar ficheiros XML
    


    # Importação de módulos
    from Bio import Entrez
    import os

    # Definimos o email para os fins de pesquisa
    Entrez.email = email
        
    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            if retmax > 1:
                print('Encontrados {} resultados em {}. Irão ser processados {} resultados.\n'.format(_['Count'],db,retmax))
            else:
                print('Encontrados {} resultados em {}. Irá ser processado 1 resultado.\n'.format(_['Count'],db))

    
    if rettype == 'fasta' or rettype == 'gb': retmax = 1

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmode=retmode, retmax=retmax)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    # print('Top 10 artigos ->',lista_ids) # Converter para uma lista de títulos mais à frente'''
    
    

    # Transfere a informação
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype=rettype,retmode=retmode)
    fetch_res = handle.read()
    
    # Usar com print para já, para não estar a bagunçar demasiado.
    # Perceber como se pode implementar a escrita de ficheiros para ser mais fácil de mastigar os resultados.

    # NÃO FUNCIONA NO GITHUB / LIVECODING
    # ADICIONAR PARAMETRO save = 'Y' SE SE PRETENDER GUARDAR FICHEIRO 
    
    if display.upper() == 'Y':
        print(fetch_res,sep='\n')


    # Verifica a condição de gravação de output como ficheiro de texto
    
    if retmode == 'xml': save = 'n'
    
    if save.upper() == 'Y':  

        # Definimos a extensão em função da base de dados e retmode
        if rettype == 'gb':
            ext = '.gbk'
        
        elif rettype == 'fasta':
            ext = '.fasta'
        
        elif retmode == 'xml':
            ext = '.xml'

        else: ext = '.txt'

        filename = term+ext

        # Verifica se existe pasta para output e cria-a caso não exista
        cwd = os.getcwd()
        outputdir = os.path.join(cwd,'output')
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)


        # Muda o working directory para a pasta de output para gerar o ficheiro
        os.chdir(outputdir)

        if os.path.isfile(filename): print('Ficheiro já existe. Não será criado novo ficheiro.')



        # Gera o ficheiro           
        with open(term+ext , 'w', encoding='utf-8') as _:
            _.write(fetch_res)

        # Retorna ao working directory anterior
        os.chdir(cwd)

        print('Ficheiro gravado.')


    handle.close()

    print('Concluído.')

    return fetch_res

In [2]:
# Exemplos da utilização da função 

email = 'pg52648@alunos.uminho.pt'
termo = 'hla-dqa1'



In [3]:
# Pesquisa 'default'
pesquisa_ncbi(email,termo)

Encontrados 2009 resultados em pubmed. Irão ser processados 10 resultados.

1. Mult Scler Relat Disord. 2023 Dec 14;82:105373. doi: 
10.1016/j.msard.2023.105373. Online ahead of print.

Gene expression in multiple sclerosis during pregnancy based on integrated 
bioinformatics analysis.

Hernández-Preciado MR(1), Torres-Mendoza BM(2), Mireles-Ramírez MA(3), 
Kobayashi-Gutiérrez A(4), Sánchez-Rosales NA(3), Vázquez-Valls E(5), 
Marquez-Pedroza J(6).

Author information:
(1)High Specialty Medical Unit, Pediatric Hospital, Western National Medical 
Center of the Mexican Institute of Social Security, Guadalajara 44340, Mexico; 
Department of Philosophical and Methodological Disciplines, University Health 
Sciences Center, University of Guadalajara, Guadalajara 44340, Mexico.
(2)Department of Philosophical and Methodological Disciplines, University Health 
Sciences Center, University of Guadalajara, Guadalajara 44340, Mexico; 
Neurosciences Division, Western Biomedical Research Center (IMSS)

"1. Mult Scler Relat Disord. 2023 Dec 14;82:105373. doi: \n10.1016/j.msard.2023.105373. Online ahead of print.\n\nGene expression in multiple sclerosis during pregnancy based on integrated \nbioinformatics analysis.\n\nHernández-Preciado MR(1), Torres-Mendoza BM(2), Mireles-Ramírez MA(3), \nKobayashi-Gutiérrez A(4), Sánchez-Rosales NA(3), Vázquez-Valls E(5), \nMarquez-Pedroza J(6).\n\nAuthor information:\n(1)High Specialty Medical Unit, Pediatric Hospital, Western National Medical \nCenter of the Mexican Institute of Social Security, Guadalajara 44340, Mexico; \nDepartment of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico.\n(2)Department of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico; \nNeurosciences Division, Western Biomedical Research Center (IMSS), Guadalajara \n44340, Mexico.\n(3)High Specialty Medical 

In [4]:
# Pesquisa de apenas 1 resultado
pesquisa_ncbi(email,termo,retmax=1)

Encontrados 2009 resultados em pubmed. Irá ser processado 1 resultado.

1. Mult Scler Relat Disord. 2023 Dec 14;82:105373. doi: 
10.1016/j.msard.2023.105373. Online ahead of print.

Gene expression in multiple sclerosis during pregnancy based on integrated 
bioinformatics analysis.

Hernández-Preciado MR(1), Torres-Mendoza BM(2), Mireles-Ramírez MA(3), 
Kobayashi-Gutiérrez A(4), Sánchez-Rosales NA(3), Vázquez-Valls E(5), 
Marquez-Pedroza J(6).

Author information:
(1)High Specialty Medical Unit, Pediatric Hospital, Western National Medical 
Center of the Mexican Institute of Social Security, Guadalajara 44340, Mexico; 
Department of Philosophical and Methodological Disciplines, University Health 
Sciences Center, University of Guadalajara, Guadalajara 44340, Mexico.
(2)Department of Philosophical and Methodological Disciplines, University Health 
Sciences Center, University of Guadalajara, Guadalajara 44340, Mexico; 
Neurosciences Division, Western Biomedical Research Center (IMSS), Gu

'1. Mult Scler Relat Disord. 2023 Dec 14;82:105373. doi: \n10.1016/j.msard.2023.105373. Online ahead of print.\n\nGene expression in multiple sclerosis during pregnancy based on integrated \nbioinformatics analysis.\n\nHernández-Preciado MR(1), Torres-Mendoza BM(2), Mireles-Ramírez MA(3), \nKobayashi-Gutiérrez A(4), Sánchez-Rosales NA(3), Vázquez-Valls E(5), \nMarquez-Pedroza J(6).\n\nAuthor information:\n(1)High Specialty Medical Unit, Pediatric Hospital, Western National Medical \nCenter of the Mexican Institute of Social Security, Guadalajara 44340, Mexico; \nDepartment of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico.\n(2)Department of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico; \nNeurosciences Division, Western Biomedical Research Center (IMSS), Guadalajara \n44340, Mexico.\n(3)High Specialty Medical 

In [5]:
# Pesquisa com output, sem display
pesquisa_ncbi(email,termo,save='y',display='n')

Encontrados 2009 resultados em pubmed. Irão ser processados 10 resultados.

Ficheiro gravado.
Concluído.


"1. Mult Scler Relat Disord. 2023 Dec 14;82:105373. doi: \n10.1016/j.msard.2023.105373. Online ahead of print.\n\nGene expression in multiple sclerosis during pregnancy based on integrated \nbioinformatics analysis.\n\nHernández-Preciado MR(1), Torres-Mendoza BM(2), Mireles-Ramírez MA(3), \nKobayashi-Gutiérrez A(4), Sánchez-Rosales NA(3), Vázquez-Valls E(5), \nMarquez-Pedroza J(6).\n\nAuthor information:\n(1)High Specialty Medical Unit, Pediatric Hospital, Western National Medical \nCenter of the Mexican Institute of Social Security, Guadalajara 44340, Mexico; \nDepartment of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico.\n(2)Department of Philosophical and Methodological Disciplines, University Health \nSciences Center, University of Guadalajara, Guadalajara 44340, Mexico; \nNeurosciences Division, Western Biomedical Research Center (IMSS), Guadalajara \n44340, Mexico.\n(3)High Specialty Medical 

In [6]:
# Pesquisa em base de dados de nucleotide/protein/gene
pesquisa_ncbi(email,termo,db='nuccore')

Encontrados 1743 resultados em nuccore. Irão ser processados 10 resultados.

LOCUS       OR913133                5683 bp    DNA     linear   PRI 25-DEC-2023
DEFINITION  Homo sapiens MHC class II antigen (HLA-DQA1) gene,
            HLA-DQA1*01:03:03extended allele, complete cds.
ACCESSION   OR913133
VERSION     OR913133.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 5683)
  AUTHORS   Kouprie,N. Sr. and Bungener,L.B.
  TITLE     Direct Submission
  JOURNAL   Submitted (07-DEC-2023) Transplant Immunologie Laboratory,
            University Medical Center Groningen, Postbus 30.001, Groningen 9700
            RB, Netherlands
COMMENT     ##Assembly-Data-START##
            Assembly Method       :: NGSengine v. 2.31.0.30527
            Sequencing Tec

'LOCUS       OR913133                5683 bp    DNA     linear   PRI 25-DEC-2023\nDEFINITION  Homo sapiens MHC class II antigen (HLA-DQA1) gene,\n            HLA-DQA1*01:03:03extended allele, complete cds.\nACCESSION   OR913133\nVERSION     OR913133.1\nKEYWORDS    .\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (bases 1 to 5683)\n  AUTHORS   Kouprie,N. Sr. and Bungener,L.B.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (07-DEC-2023) Transplant Immunologie Laboratory,\n            University Medical Center Groningen, Postbus 30.001, Groningen 9700\n            RB, Netherlands\nCOMMENT     ##Assembly-Data-START##\n            Assembly Method       :: NGSengine v. 2.31.0.30527\n            Sequencing Technology :: Illumina\n            ##Assembly-Data-END##\nF

In [7]:
# Identificado o identificador (OR913133 neste caso) transferimos o ficheiro .GBK para processamento
# Refinamos a pesquisa na base de dados gene antes de transferir o ficheiro GBK

pesquisa_ncbi(email,'OR913133',db='nuccore',rettype='gb',retmax=1,save='y',display='n')

Encontrados 1 resultados em nuccore. Irá ser processado 1 resultado.

Ficheiro gravado.
Concluído.


'LOCUS       OR913133                5683 bp    DNA     linear   PRI 25-DEC-2023\nDEFINITION  Homo sapiens MHC class II antigen (HLA-DQA1) gene,\n            HLA-DQA1*01:03:03extended allele, complete cds.\nACCESSION   OR913133\nVERSION     OR913133.1\nKEYWORDS    .\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (bases 1 to 5683)\n  AUTHORS   Kouprie,N. Sr. and Bungener,L.B.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (07-DEC-2023) Transplant Immunologie Laboratory,\n            University Medical Center Groningen, Postbus 30.001, Groningen 9700\n            RB, Netherlands\nCOMMENT     ##Assembly-Data-START##\n            Assembly Method       :: NGSengine v. 2.31.0.30527\n            Sequencing Technology :: Illumina\n            ##Assembly-Data-END##\nF

In [8]:
# Pesquisamos agora pela proteína para preparar os ficheiros para a análise da mesma

pesquisa_ncbi(email,'HLA-DQA1',db='protein')

# Encontramos o identificador WQB05986, refinamos a pesquisa

Encontrados 1521 resultados em protein. Irão ser processados 10 resultados.

LOCUS       WQB05986                 255 aa            linear   PRI 20-DEC-2023
DEFINITION  MHC class II antigen [Homo sapiens].
ACCESSION   WQB05986
VERSION     WQB05986.1
DBSOURCE    accession OR921180.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (residues 1 to 255)
  AUTHORS   Deghaide,N.H.S., Martins,G.V., Domeneghetti,M.S., Correa,A.P.B.,
            Santos,V.T. and Donadi,E.A.
  TITLE     Direct Submission
  JOURNAL   Submitted (07-DEC-2023) Laboratorio de HLA, Hemocentro de Ribeirao
            Preto, Rua: Tenente Catao Roxo, Ribeirao Preto, SP 14051-140,
            Brazil
FEATURES             Location/Qualifiers
     source          1..255
                     /organism

'LOCUS       WQB05986                 255 aa            linear   PRI 20-DEC-2023\nDEFINITION  MHC class II antigen [Homo sapiens].\nACCESSION   WQB05986\nVERSION     WQB05986.1\nDBSOURCE    accession OR921180.1\nKEYWORDS    .\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (residues 1 to 255)\n  AUTHORS   Deghaide,N.H.S., Martins,G.V., Domeneghetti,M.S., Correa,A.P.B.,\n            Santos,V.T. and Donadi,E.A.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (07-DEC-2023) Laboratorio de HLA, Hemocentro de Ribeirao\n            Preto, Rua: Tenente Catao Roxo, Ribeirao Preto, SP 14051-140,\n            Brazil\nFEATURES             Location/Qualifiers\n     source          1..255\n                     /organism="Homo sapiens"\n                     /db_xref="taxon:96

In [9]:
# Guardamos o ficheiro FASTA
pesquisa_ncbi(email,'WQB05986',db='protein',retmax=1,rettype='fasta',save='Y',display='n')

Encontrados 1 resultados em protein. Irá ser processado 1 resultado.

Ficheiro gravado.
Concluído.


'>WQB05986.1 MHC class II antigen [Homo sapiens]\nMILNKALLLGALALTTVMSPCGGEDIVADHVASCGVNLYQFYGPSGQYTHEFDGDEQFYVDLERKETAWR\nWPEFSKFGGFDPQGALRNMAVAKHNLNIMIKRYNSTAATNEVPEVTVFSKSPVTLGQPNTLICLVDNIFP\nPVVNITWLSNGQSVTEGVSETSFLSKSDHSFFKISYLTFLPSAGEIYDCKVEHWGLDQPLLKHWEPEIPA\nPMSELTETVVCALGLSVGLMGIVVGTVFIIQGLRSVGASRHQGPL\n\n'

In [10]:
pesquisa_ncbi(email,'DQA1_HUMAN',db='protein')

Encontrados 1 resultados em protein. Irão ser processados 10 resultados.

LOCUS       DQA1_HUMAN               254 aa            linear   PRI 08-NOV-2023
DEFINITION  RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1
            chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha;
            AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags:
            Precursor.
ACCESSION   P01909
VERSION     P01909.1
DBSOURCE    UniProtKB: locus DQA1_HUMAN, accession P01909;
            class: standard.
            extra
            accessions:O19630,O19706,P01907,P01908,P04225,P04226,P05536,P79553,
            Q06751,Q29876,Q29994,Q2Q6Y6,Q2Q6Y7,Q2Q6Y8,Q2WCM3,Q30064,Q30067,
            Q30068,Q30070,Q30071,Q30072,Q30073,Q30086,Q30101,Q5Y7D5,Q5Y7F5,
            Q6ICU6,Q6PR46,Q6QDB1,Q860W2,Q860W4,Q9BD37,Q9TPM3,Q9UM31
            created: Jul 21, 1986.
            sequence updated: Jul 21, 1986.
            annotation updated: Nov 8, 2023.
            xrefs: X00033

'LOCUS       DQA1_HUMAN               254 aa            linear   PRI 08-NOV-2023\nDEFINITION  RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1\n            chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha;\n            AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags:\n            Precursor.\nACCESSION   P01909\nVERSION     P01909.1\nDBSOURCE    UniProtKB: locus DQA1_HUMAN, accession P01909;\n            class: standard.\n            extra\n            accessions:O19630,O19706,P01907,P01908,P04225,P04226,P05536,P79553,\n            Q06751,Q29876,Q29994,Q2Q6Y6,Q2Q6Y7,Q2Q6Y8,Q2WCM3,Q30064,Q30067,\n            Q30068,Q30070,Q30071,Q30072,Q30073,Q30086,Q30101,Q5Y7D5,Q5Y7F5,\n            Q6ICU6,Q6PR46,Q6QDB1,Q860W2,Q860W4,Q9BD37,Q9TPM3,Q9UM31\n            created: Jul 21, 1986.\n            sequence updated: Jul 21, 1986.\n            annotation updated: Nov 8, 2023.\n            xrefs: X00033.1, CAA24917.1, X00370.1, X00452.1, CAA25141.1,\n       